## Entrenamiento del modelo

### 0 - Configuracion Sagemaker

In [ ]:
import sagemaker
from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer

In [ ]:
# identificación de región
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

# indetificación de rol
role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

# sesión de sagemaker
session = sagemaker.Session()

### 1 - Selección y parametrización del modelo

In [ ]:
# Necesitamos el container del modelo xgboost. El nombre del contenedor cambia según región y modelo
# para ayudarnos tenemos la función get_image_uri
container = retrieve('xgboost', region, version='latest')

In [ ]:
s3_folder_bucket = "clasificador_sentimiento"

In [ ]:
# primero creamos el estimador
xgb = sagemaker.estimator.Estimator(container,  # uri del contenedor
                                    role,
                                    # base_job_name='modelo-xgb',
                                    volume_size=8,
                                    instance_count=1,  # número de instancias
                                    instance_type='ml.m4.xlarge',  # tipo de instancias
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), s3_folder_bucket), # almacenamiento de los resultados
                                    sagemaker_session=session)

In [ ]:
print(f"URI de los resultados: {'s3://{}/{}/output'.format(session.default_bucket(), s3_folder_bucket)}")

In [ ]:
# parametrización del modelo
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=12,
                        num_round=500)

### 2 - Entrenamiento

In [ ]:
# resultado del notebook anterior
inputs= {}

In [ ]:
# cargamos los datos
s3_input_train = sagemaker.TrainingInput(s3_data=inputs['train'], content_type='csv')
s3_input_validation = sagemaker.TrainingInput(s3_data=inputs['validation'], content_type='csv')
s3_input_test = sagemaker.TrainingInput(s3_data=inputs['test'], content_type='csv')

In [ ]:
# entrenamiento
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

### 3 - Despliegue y creación de un endpoint

#### A - Despliegue Modelo Serverless

Solo pagamos por el uso que realizamos del modelo

In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

In [ ]:
# configuración del endpoint
serverless_config = ServerlessInferenceConfig(
  memory_size_in_mb=1024,
  max_concurrency=5,
)

In [ ]:
# despliegue
xgb_predictor = xgb.deploy(endpoint_name='mi-primer-endpoint-serverless',
                           serverless_inference_config=serverless_config,
                           serializer=CSVSerializer())

#### B - Despliegue Máquina Virtual

Mejor realizar el taller con la configuración serverless para evitar costes inesperados. Si quieres probar, aquí tienes el código para desplegar el modelo en una instancia. Nos facturan por el tiempo total que tengamos la máquina virtual encendida independientemente del uso que hagamos de ella.

In [ ]:
# xgb_predictor = xgb.deploy(endpoint_name='mi-primer-endpoint',
#                            initial_instance_count=1,
#                            instance_type='ml.m4.xlarge',
#                            serializer=CSVSerializer())

### 4 - Optimización de hiperparámetros
<br> Búsqueda por cuadrícula (Grid Search)

In [ ]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

xgb_hyperparameter_tuner = HyperparameterTuner(estimator = xgb, # El estimador
                                               objective_metric_name = 'validation:rmse', # Métrica para comparar los modelos
                                               objective_type = 'Minimize', # Minimizar o maximiza  la métrica objetivo
                                               max_jobs = 4, # Número total de modelos a entrenar
                                               max_parallel_jobs = 2, # Número máximo de modelos en paralelo a entrenar
                                               hyperparameter_ranges = {
                                                    'max_depth': IntegerParameter(3, 12),
                                                    'eta'      : ContinuousParameter(0.05, 0.5),
                                                    'min_child_weight': IntegerParameter(2, 8),
                                                    'subsample': ContinuousParameter(0.5, 0.9),
                                                    'gamma': ContinuousParameter(0, 10),
                                               })

In [ ]:
# xgb_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})